In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cirq

In [ ]:
from cirq.noise.test import FluxNoiseModel

model = FluxNoiseModel(0.05)

model.seen()

In [ ]:
# 定义三比特
q0, q1, q2 = cirq.LineQubit.range(3)

# 构造电路
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.X(q1),            
    cirq.CX(q0, q1),
    cirq.X(q0),        
    cirq.X(q1),        
    cirq.CX(q1, q2),
    cirq.X(q2),        
    cirq.measure(q0, q1, q2)  
)

print(circuit)
circuit.all_qubits()


In [ ]:
{cirq.OpIdentifier(cirq.X, q) : cirq.bit_flip(0.1).on(q) for q in circuit._all_qubits}

In [ ]:
from cirq.devices import InsertionNoiseModel

target_op = cirq.OpIdentifier(cirq.X, q1)
insert_op = cirq.bit_flip(0.1).on(q1)
ops_added={cirq.OpIdentifier(cirq.X, q) : cirq.bit_flip(0.1).on(q) for q in circuit.all_qubits()}|{cirq.OpIdentifier(cirq.X, q): cirq.depolarize(0.1).on(q) for q in circuit.all_qubits()}
noise_model = InsertionNoiseModel(
    ops_added=ops_added,
    require_physical_tag = False,
)
# noise_model = InsertionNoiseModel(
#     ops_added={cirq.OpIdentifier(cirq.X, q1): cirq.bit_flip(0.1).on(q1),
#                cirq.OpIdentifier(cirq.X, q2): cirq.bit_flip(0.1).on(q2)
#                },
#     require_physical_tag=False,
# )

noisy_circuit = circuit.with_noise(noise_model)
print(noisy_circuit)

In [ ]:
{q:q for q in [1, 2, 3]}

#### Noisy GHZ

In [ ]:
import cirq

# 定义三比特
q0, q1, q2 = cirq.LineQubit.range(3)

# 构造 GHZ 电路
circuit = cirq.Circuit(
    cirq.H(q0),             # 让第一个量子比特形成叠加
    cirq.CX(q0, q1),        # 纠缠第二个量子比特
    cirq.CX(q1, q2),        # 纠缠第三个量子比特
    cirq.measure(q0, q1, q2, key='m')  # 测量
)

print(circuit)


In [ ]:
noise_model = cirq.ConstantQubitNoiseModel(cirq.depolarize(p=0.01))

simulator = cirq.DensityMatrixSimulator(noise=noise_model)

result = simulator.run(circuit, repetitions=1000)
print(result.histogram(key='m'))


#### test multi

In [ ]:
import cirq

q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)

qubits = len(circuit.all_qubits())

print(f"电路包含 {qubits} 个量子比特")


In [ ]:
from cirq.noise import MultiInsertionNoiseModel

def my_noise_rule(op):
    pre_ops = []
    post_ops = []
    if isinstance(op.gate, cirq.XPowGate) or isinstance(op.gate, cirq.CXPowGate):
        for q in op.qubits:
            pre_ops.append(cirq.bit_flip(0.1).on(q))
            post_ops.append(cirq.depolarize(0.05).on(q))
    return pre_ops, post_ops


noise_model = MultiInsertionNoiseModel(noise_rule=my_noise_rule)

noisy_circuit = circuit.with_noise(noise_model)
print(noisy_circuit)


In [ ]:
sim = cirq.DensityMatrixSimulator(noise=MultiInsertionNoiseModel(noise_rule=my_noise_rule))

result = sim.run(circuit, repetitions=1000)
print(result.histogram(key='m'))


#### test idle

In [ ]:
import cirq
from cirq.noise import IdleNoiseModel

# 构建电路
q0, q1, q2, q3 = cirq.LineQubit.range(4)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.X(q2),
    cirq.CX(q0, q1),
    cirq.H(q3),
    cirq.X(q3),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, q3, key='m')
)

# 创建 noise model
noise_model = IdleNoiseModel(T1=30, Tphi=50, duration=1)  # 单位一致：如微秒

# 执行
sim = cirq.DensityMatrixSimulator(noise=noise_model)
result = sim.run(circuit, repetitions=1000)

# 打印结果
print(circuit)
print(circuit.moments)
print(circuit.with_noise(noise_model))
print(result.histogram(key='m'))


In [ ]:
print(circuit.with_noise(noise_model).moments)

#### test photon decay

In [ ]:
import cirq
from cirq.noise import PhotonDecayNoiseModel

# 构建电路
q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)


photon_noise = PhotonDecayNoiseModel()
sim = cirq.DensityMatrixSimulator(noise=photon_noise)
result = sim.run(circuit, repetitions=1000)

print(circuit.with_noise(photon_noise))
print(result.histogram(key='m'))




In [ ]:
print(circuit.moments)

circuit.moments[1].operations[0]

# type(circuit.moments[0].operations[0])

# PhotonDecayNoiseModel().noisy_operation(circuit.moments[0].operations[0])

#### test Ry

In [ ]:
import cirq
import numpy as np
from cirq.noise import RyGateNoiseModel

# 构建电路
q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)

model = RyGateNoiseModel()
sim = cirq.DensityMatrixSimulator(noise=model)

# q = cirq.LineQubit(0)
# circuit = cirq.Circuit(
#     cirq.ry(np.pi/2).on(q),
#     cirq.measure(q, key='m')
# )

result = sim.run(circuit, repetitions=1000)
print(circuit.with_noise(RyGateNoiseModel()))
print(result.histogram(key='m'))


In [ ]:
ncircuit = circuit.with_noise(RyGateNoiseModel())
for i, moment in enumerate(ncircuit.moments):
    for j, op in enumerate(moment.operations):
        print(f"moment[{i}].operations[{j}]: {op}")


In [ ]:
ncircuit.moments[0]

#### test flux sample

In [ ]:
import cirq
from cirq.noise import FluxNoiseContext
from cirq.noise import SampleFluxNoiseModel

q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)

context = FluxNoiseContext(phi_rms=0.01, qubits=[q0, q1, q2])
context.generate()

model = SampleFluxNoiseModel(context)
sim = cirq.DensityMatrixSimulator(noise=model)

result = sim.run(circuit, repetitions=1000)

print(circuit.with_noise(model))
print(result.histogram(key='m'))



#### test flux average

#### test composite

In [ ]:
# import cirq
# from typing import List, Sequence, Generator, Union, Iterable

# def flatten_ops(seq: Sequence[Union[cirq.Operation, Sequence]]) -> Generator[cirq.Operation, None, None]:
#     """Recursively flatten a sequence of Operations and nested Sequences."""
#     for item in seq:
#         if isinstance(item, cirq.Operation):
#             yield item
#         elif isinstance(item, Sequence):
#             yield from flatten_ops(item)
#         else:
#             raise TypeError(f"Expected cirq.Operation or Sequence, got {type(item)}: {item}")

# def split_op_list(op_list, target_op):
#     if target_op not in op_list:
#         return [], op_list  # 全部是前部分，后面是空

#     index = op_list.index(target_op)
#     part1 = op_list[ : index ]  
#     part2 = op_list[index + 1 : ]  
#     return part1, part2

# class CompositeNoiseModel(cirq.NoiseModel):

#     def __init__(self, noise_models: List[cirq.NoiseModel]):
#         self.op_models = []
#         self.moment_models = []

#         for model in noise_models:
#             if hasattr(model, 'noise_model_type'):
#                 if model.noise_model_type() == 'operation':
#                     self.op_models.append(model)
#                 elif model.noise_model_type() == 'moment':
#                     self.moment_models.append(model)
#                 else:
#                     raise ValueError(f"Unknown noise model type declared: {model}")
#             else:
#                 raise ValueError(f"Noise model {model} missing noise_model_type declaration.")
            
#     def print_models(self):
#         print("Operation Models:")
#         for model in self.op_models:
#             print(f" - {model}")
#         print("Moment Models:")
#         for model in self.moment_models:
#             print(f" - {model}")

#     def noisy_moment(
#         self, moment: Iterable[cirq.Moment], system_qubits: Sequence[cirq.Qid]
#     ) -> Sequence[cirq.OP_TREE]:
#         final_OP_TREE = [] # OP_TREE to return
#         final_ops_left = []
#         final_ops_right = []
#         origin_moment = moment
#         final_moments = [origin_moment]
#         ops = moment.operations

#         for op in ops:
#             new_ops_left = []
#             new_ops_right = []

#             # 对该 op 施加所有 op 级 model, 得到新的 ops 列表
#             for model in self.op_models:

#                 noisy_ops = model.noisy_operation(op)
#                 noisy_left, noisy_right = split_op_list(noisy_ops, op)

#                 new_ops_left = new_ops_left + noisy_left
#                 new_ops_right = noisy_right + new_ops_right

#             final_ops_left.extend(new_ops_left)
#             final_ops_right.extend(new_ops_right)

#         # 处理 moment_models
#         for model in self.moment_models:

#             final_moments = model.noisy_moment(origin_moment, system_qubits)
#             filtered = []
#             for m in final_moments:
#                 if isinstance(m, cirq.Moment) and m == origin_moment:
#                     continue
#                 filtered.append(m)
                
#         final_OP_TREE = final_ops_left + final_moments + final_ops_right

#         return final_OP_TREE



In [ ]:
import cirq
from cirq.noise import FluxNoiseContext, IdleNoiseModel, PhotonDecayNoiseModel, SampleFluxNoiseModel, CompositeNoiseModel

q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.X(q2),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)

model1 = PhotonDecayNoiseModel()
context = FluxNoiseContext(phi_rms=0.01, qubits=[q0, q1, q2])
context.generate()
model2 = SampleFluxNoiseModel(context)
model3 = IdleNoiseModel()

composite_model = CompositeNoiseModel([model1, model2, model3])

sim = cirq.DensityMatrixSimulator(noise=composite_model)
result = sim.run(circuit, repetitions=1000)

print(circuit.with_noise(composite_model))
#print(circuit.with_noise(composite_model).moments)
print(result.histogram(key='m'))
#print(circuit.moments)
#composite_model.print_models()

In [ ]:
hasattr(IdleNoiseModel(), 'noisy_operation') and callable(getattr(IdleNoiseModel, 'noisy_operation'))

In [ ]:
import cirq

def has_overridden_noisy_operation(model):
    return model.__class__.noisy_operation != cirq.NoiseModel.noisy_operation

def has_overridden_noisy_moment(model):
    return model.__class__.noisy_moment != cirq.NoiseModel.noisy_moment

# 测试模型
from cirq.noise import PhotonDecayNoiseModel, IdleNoiseModel

model1 = PhotonDecayNoiseModel()
model2 = IdleNoiseModel()

print(f"PhotonDecayNoiseModel overrides noisy_operation? {has_overridden_noisy_operation(model1)}")
print(f"PhotonDecayNoiseModel overrides noisy_moment? {has_overridden_noisy_moment(model1)}")

print(f"IdleNoiseModel overrides noisy_operation? {has_overridden_noisy_operation(model2)}")
print(f"IdleNoiseModel overrides noisy_moment? {has_overridden_noisy_moment(model2)}")


In [ ]:
import cirq

def is_method_overridden(instance, method_name):
    cls = instance.__class__
    base = cirq.NoiseModel

    method = getattr(cls, method_name, None)
    base_method = getattr(base, method_name, None)

    if method is None or base_method is None:
        return False

    # 解引用 function 本体再比较
    return getattr(method, '__func__', method) is not getattr(base_method, '__func__', base_method)


In [ ]:
from cirq.noise import PhotonDecayNoiseModel, IdleNoiseModel

model1 = PhotonDecayNoiseModel()
model2 = IdleNoiseModel()

print("PhotonDecayNoiseModel:")
print("  noisy_operation overridden?", is_method_overridden(model1, 'noisy_operation'))
print("  noisy_moment overridden?", is_method_overridden(model1, 'noisy_moment'))

print("IdleNoiseModel:")
print("  noisy_operation overridden?", is_method_overridden(model2, 'noisy_operation'))
print("  noisy_moment overridden?", is_method_overridden(model2, 'noisy_moment'))


In [ ]:
import inspect

def list_class_methods(cls):
    methods = inspect.getmembers(cls, predicate=inspect.isfunction)
    for name, _ in methods:
        print(name)

In [ ]:
import cirq
from cirq.noise import PhotonDecayNoiseModel

list_class_methods(PhotonDecayNoiseModel)


#### cirq诗人握持啊

In [ ]:
print(circuit)

In [ ]:
print(circuit.with_noise(PhotonDecayNoiseModel()))

In [ ]:
ncircuit = circuit.with_noise(PhotonDecayNoiseModel())
for moment in ncircuit.moments:
    for op in moment.operations:
        print(f"moment[{ncircuit.moments.index(moment)}].operations[{moment.operations.index(op)}]: {op}")

In [ ]:
for moment in circuit.moments:
    for op in moment.operations:
        print(f"moment[{circuit.moments.index(moment)}].operations[{moment.operations.index(op)}]: {op}")

#### test OP-Tree

In [ ]:
op1 = cirq.X(q0)
op2 = cirq.Y(q1)
op3 = cirq.Z(q2)



In [ ]:
test_trees = [
    op1,
    [op1, op2],
    [[op1], [op2]],
    [cirq.Moment([op1, op2])],
    [cirq.Moment([op1]), cirq.Moment([op2])],
    [[op1, op2], [op3]],
]



In [ ]:
for tree in test_trees:
    print("Testing OP_TREE:", tree)
    c = cirq.Circuit(tree)
    print(c)

In [ ]:
import cirq
from cirq.noise import NamedGate

q0, q1, q2, q3, q4, q5 = cirq.LineQubit.range(6)
A_q2 = NamedGate('A', num_qubits = 1).on(q2)

B_q0 = NamedGate('B', num_qubits = 1).on(q0)
B_q1 = NamedGate('B', num_qubits = 1).on(q1)
B_q2 = NamedGate('B', num_qubits = 1).on(q2)
B_q3 = NamedGate('B', num_qubits = 1).on(q3)
B_q4 = NamedGate('B', num_qubits = 1).on(q4)
B_q5 = NamedGate('B', num_qubits = 1).on(q5)

C_q0 = NamedGate('C', num_qubits = 1).on(q0)
C_q1 = NamedGate('C', num_qubits = 1).on(q1)
C_q2 = NamedGate('C', num_qubits = 1).on(q2)
C_q3 = NamedGate('C', num_qubits = 1).on(q3)
C_q4 = NamedGate('C', num_qubits = 1).on(q4)
C_q5 = NamedGate('C', num_qubits = 1).on(q5)

D_q0 = NamedGate('D', num_qubits = 1).on(q0)
D_q1 = NamedGate('D', num_qubits = 1).on(q1)
D_q2 = NamedGate('D', num_qubits = 1).on(q2)
D_q3 = NamedGate('D', num_qubits = 1).on(q3)
D_q4 = NamedGate('D', num_qubits = 1).on(q4)
D_q5 = NamedGate('D', num_qubits = 1).on(q5)

D_q134 = NamedGate('D', num_qubits = 3).on(q1, q3, q4)

E_q34 = NamedGate('E', num_qubits = 2).on(q3, q4)

op0 = NamedGate('op0', num_qubits = 1).on(q0)
op1 = NamedGate('op1', num_qubits = 1).on(q1)
op2 = NamedGate('op2', num_qubits = 1).on(q2)
op3 = NamedGate('op3', num_qubits = 1).on(q3)
op4 = NamedGate('op4', num_qubits = 1).on(q4)
op5 = NamedGate('op5', num_qubits = 1).on(q5)

op34 = NamedGate('op34', num_qubits = 2).on(q3, q4)
op012345 = NamedGate('op012345', num_qubits = 6).on(q0, q1, q2, q3, q4, q5)

In [ ]:
tree = [[C_q0, C_q1, D_q134, C_q5],
        [op012345],
        [D_q0, D_q1, D_q3],
        [C_q5],
        [C_q0, C_q1, E_q34, C_q5],
        [C_q0, C_q1, B_q3, B_q4, B_q5],
        [B_q0, B_q1]
        ]
c = cirq.Circuit(tree)
print(c)

In [ ]:
tree = [[C_q0, C_q1, C_q5, op012345, D_q0, D_q1, D_q3, C_q5, C_q0, C_q1, E_q34, C_q5, C_q0, C_q1, B_q3, B_q4, B_q5, B_q0, B_q1],
        [],
        [],
        ]
c = cirq.Circuit(tree)
print(c)

In [ ]:
tree = [[C_q0, C_q1, C_q5, cirq.Moment(op0, op1, op34, op5)],
        #cirq.Moment(), 
        cirq.Moment([A_q2]),
        #cirq.Moment(),
        [D_q0, D_q1, D_q3, C_q5, C_q5],
        [C_q0, C_q1, E_q34, C_q5, C_q0, C_q1, B_q3, B_q4, B_q5, B_q0, B_q1],
        ]
c = cirq.Circuit(tree)
print(c)
print(c.moments)

In [ ]:
tree = [[C_q0, C_q1, C_q5, cirq.Moment(op0, op1, op34, op5)],
        #cirq.Moment(), 
        [cirq.Moment([A_q2])],
        #cirq.Moment(),
        cirq.Moment([D_q0, D_q1, D_q3, C_q5]),
        [C_q0, C_q1, E_q34, C_q5, C_q0, C_q1, B_q3, B_q4, B_q5, B_q0, B_q1],
        ]
c = cirq.Circuit(tree)
print(c)

#### test flux average

In [ ]:
import cirq
from cirq.noise import AverageFluxNoiseModel

q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)
circuit_no_measure = cirq.Circuit(op for op in circuit.all_operations() if not isinstance(op.gate, cirq.MeasurementGate))

avg_flux = AverageFluxNoiseModel()
sim = cirq.DensityMatrixSimulator(noise=avg_flux)
result = sim.run(circuit, repetitions=1000)
rho = sim.simulate(circuit_no_measure).final_density_matrix

print(circuit.with_noise(avg_flux))
print(result.histogram(key='m'))
print(f"Final density matrix: {rho}")


sim_ideal = cirq.DensityMatrixSimulator(noise=None)
rho_ideal = sim_ideal.simulate(circuit_no_measure).final_density_matrix
print(f"Ideal final density matrix: {rho_ideal}")

#### gate fidelity

In [ ]:

def compute_final_state_fidelity(circuit, noise_model=None):
    """运行电路并计算最终态保真度"""
    # 获取理想最终态
    sim_ideal = cirq.DensityMatrixSimulator()
    rho_ideal = sim_ideal.simulate(circuit).final_density_matrix

    # 获取带噪声最终态
    sim_noisy = cirq.DensityMatrixSimulator(noise=noise_model)
    rho_noisy = sim_noisy.simulate(circuit).final_density_matrix

    # 保真度计算
    fidelity = cirq.fidelity(rho_ideal, rho_noisy)

    return fidelity, rho_ideal, rho_noisy

In [ ]:
# fidelity, rho_ideal, rho_noisy = compute_final_state_fidelity(circuit, noise_model = avg_flux)

sim_noisy = cirq.DensityMatrixSimulator(noise = AverageFluxNoiseModel())
rho_noisy = sim_noisy.simulate(circuit)

print(rho_noisy)

In [ ]:
q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q1, q2),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)
for i, moment in enumerate(circuit.moments):
    print(f"Moment {i}: {moment}")

In [ ]:
circuit.moments[0].operations[1]

In [ ]:
circuit.moments[2].operations[0]

In [ ]:
circuit.moments[0].operations[1] is circuit.moments[2].operations[0]

In [ ]:
circuit.moments[0].operations[1] == circuit.moments[2].operations[0]

#### comprehensive

In [ ]:
import cirq
from cirq.noise import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, SampleFluxNoiseModel, AverageFluxNoiseModel, CompositeNoiseModel, FluxNoiseContext

q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)
circuit_no_measure = cirq.Circuit(op for op in circuit.all_operations() if not isinstance(op.gate, cirq.MeasurementGate))

context = FluxNoiseContext(phi_rms=0.01, qubits=[q0, q1, q2])
context.generate()

model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel()
model3 = RyGateNoiseModel()
model4 = SampleFluxNoiseModel(context)
model5 = AverageFluxNoiseModel()

composite_model_sample = CompositeNoiseModel([model1, model2, model3, model4])
composite_model_avg = CompositeNoiseModel([model1, model2, model3, model5])

sim_sample = cirq.DensityMatrixSimulator(noise=composite_model_sample)
result_sample = sim_sample.run(circuit, repetitions=1000)
rho_sample = sim_sample.simulate(circuit_no_measure).final_density_matrix

sim_avg = cirq.DensityMatrixSimulator(noise=composite_model_avg)
result_avg = sim_avg.run(circuit, repetitions=1000)
rho_avg = sim_avg.simulate(circuit_no_measure).final_density_matrix

sim_ideal = cirq.DensityMatrixSimulator(noise=None)
rho_ideal = sim_ideal.simulate(circuit_no_measure).final_density_matrix

print("ideal circuit: \n" ,circuit.with_noise(None))
print("circuit with sample noise model: \n", circuit.with_noise(composite_model_sample))
# print("circuit with average noise model: \n", circuit.with_noise(composite_model_avg))

# print("Sample noise model result histogram:", result_sample.histogram(key='m'))
# print("Average noise model result histogram:", result_avg.histogram(key='m'))
# print("Final density matrix with sample noise model:\n", rho_sample)
# print("Final density matrix with average noise model:\n", rho_avg)
# print("Ideal final density matrix:\n", rho_ideal)
cirq.fidelity(rho_ideal, rho_sample, qid_shape = (2, 2, 2))

#### test time_anno

In [ ]:
import cirq
from cirq.noise.models import SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils import assign_timed_circuit_context

# ---------- Step 1: 构建原始电路 ----------
q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q1, q2),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.measure(q0, q1, q2, key='m')
)
print(f"Original circuit: \n{circuit}")

# ---------- Step 2: 分配时间信息 ----------
ctx = assign_timed_circuit_context(circuit, tau_c=60)
ctx.print_timing_map()


# ---------- Step 3: 采样 δϕ 序列 ----------
# 获取所有 qubits + 最大 segment ID
all_qubits = sorted(circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q)))
max_seg = max(info.segment_id for infos in ctx.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(all_qubits, max_seg)

delta_phis = sample_flux_noise_segments(all_qubits, max_seg)

# ---------- Step 4: 创建 SegmentedNoiseModel ----------
noise_model = SegmentedFluxNoiseModel(ctx.timing_map, delta_phis)

# ---------- Step 5: 仿真器执行 ----------
simulator = cirq.DensityMatrixSimulator(noise=noise_model)
noisy_circuit = circuit.with_noise(noise_model)
print(f"\nNoisy circuit:\n{noisy_circuit}")
# noise_model.reset()
# result = simulator.run(noisy_circuit, repetitions=1000)
# print(f"\nSimulation results:\n{result.histogram(key='m')}")

In [ ]:
import cirq
from cirq.noise import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, CompositeNoiseModel

# ---------- Step 6: 复合模型 ----------

composite_model = CompositeNoiseModel([
    IdleNoiseModel(),
    PhotonDecayNoiseModel(),
    RyGateNoiseModel(),
    noise_model
])

simulator = cirq.DensityMatrixSimulator(noise=composite_model)
composite_noisy_circuit = circuit.with_noise(composite_model)
print(f"\nComposite noisy circuit:\n{composite_noisy_circuit}")
result = simulator.run(composite_noisy_circuit, repetitions=1000)
print(f"\nSimulation results:\n{result.histogram(key='m')}")

#### test metric

In [ ]:
import cirq
from cirq.noise import IdleNoiseModel, fidelity, trace_distance
# 定义三比特
q0, q1, q2 = cirq.LineQubit.range(3)
circuit_no_measure = cirq.Circuit(
        cirq.H(q0),
        cirq.CX(q0, q1),
        cirq.CX(q1, q2)
    )

noise_model = IdleNoiseModel()

simulator1 = cirq.DensityMatrixSimulator()
simulator2 = cirq.DensityMatrixSimulator(noise=noise_model)
rho = simulator1.simulate(circuit_no_measure).final_density_matrix
phi = simulator2.simulate(circuit_no_measure).final_density_matrix

print(rho)
print(phi)

print("Fidelity:", fidelity(rho, phi))
print("Trace Distance(eig):", trace_distance(rho, phi))
print("Trace Distance(svd):", trace_distance(rho, phi, type='svd'))

In [ ]:
import cirq

q0, q1 = cirq.LineQubit.range(2)

print([name for name in dir(cirq.SingleQubitCliffordGate)])


#### test exp1

In [ ]:
from cirq.noise import run_example

result = run_example("../dev/Cirq/cirq-core/cirq/noise/configs/example.yaml")

#### test QSIH circuit

In [ ]:
import cirq
import numpy as np
import sympy

def U_block(q0, q1, θx, θy, θz):
    """定义两量子比特参数化门 U(θ)，作用在 q0, q1 上"""
    return [
        cirq.CNOT(q0, q1),
        cirq.rz(θz)(q0),
        cirq.H(q1),
        cirq.rz(θx + np.pi/2)(q1),
        cirq.CNOT(q0, q1),
        cirq.rz(-θy)(q0),
        cirq.H(q1),
        cirq.CNOT(q0, q1),
        cirq.rx(np.pi/2)(q0),      # √X
        cirq.rx(-np.pi/2)(q1),     # √X†
    ]

def add_U_layer(circuit, qubits, θs, even_layer=True):
    """为给定线路添加一层 U(θ) 门(交错结构), even_layer 控制是否为偶数层(从第0层开始)"""
    n = len(qubits)
    start = 0 if even_layer else 1
    for i in range(start, n - 1, 2):
        θx, θy, θz = θs[i]
        circuit.append(U_block(qubits[i], qubits[i + 1], θx, θy, θz))

n_qubits = 6
n_layers = 4

qubits = cirq.LineQubit.range(n_qubits)
circuit = cirq.Circuit()

# 创建符号参数
θ_symbols = {
    (layer, i): (
        sympy.Symbol(f'theta_x_{layer}_{i}'),
        sympy.Symbol(f'theta_y_{layer}_{i}'),
        sympy.Symbol(f'theta_z_{layer}_{i}'),
    )
    for layer in range(n_layers)
    for i in range(n_qubits - 1)
}

# 添加层
for layer in range(n_layers):
    θs = [θ_symbols[(layer, i)] for i in range(n_qubits - 1)]
    add_U_layer(circuit, qubits, θs, even_layer=(layer % 2 == 0))

print(circuit)


#### test exp1_new

In [ ]:
import yaml
from experiments import make_run_dir, run_exp1_experiment, save_results_exp1, load_config

# 1) 读取 YAML
cfg = load_config("exp1_fid.yaml")


# 2) 跑实验（返回结果 + 元数据）
results, meta = run_exp1_experiment(cfg)


run_dir = make_run_dir(exp_name="experiment1")
# 3) 保存结果（带 header）
paths = save_results_exp1(
    results, meta, run_dir,                 # 有就传
    results_filename="results.txt",
    samples_filename="samples.tsv",
    circuit_filename="circuit.txt",
)


#### test exp2

In [ ]:
import cirq
import numpy as np

mat = np.random.rand(4,4)

mat[0, 0]


In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# 模型: y = A * p^m + B
def model(m, A, p, B):
    return A * (p ** m) + B

# 生成模拟数据
np.random.seed(42)
m_list = np.arange(1, 21)                     # RB序列长度
A_true, p_true, B_true = 0.9, 0.98, 0.25      # 真值
y_true = model(m_list, A_true, p_true, B_true)
y_noisy = y_true + 0.01 * np.random.randn(len(y_true))  # 加噪声

# 初值 + 边界
p0 = [0.8, 0.99, 0.3]
bounds = ([0.0, 0.0, 0.0], [1.0, 1.0, 1.0])

# 拟合
popt, pcov = curve_fit(model, m_list, y_noisy, p0=p0, bounds=bounds)

# 拟合结果
A_fit, p_fit, B_fit = popt
A_std, p_std, B_std = np.sqrt(np.diag(pcov))
print(f"A = {A_fit:.4f} ± {A_std:.4f}")
print(f"p = {p_fit:.4f} ± {p_std:.4f}")
print(f"B = {B_fit:.4f} ± {B_std:.4f}")

# 可视化
plt.scatter(m_list, y_noisy, label="data", color="tab:blue")
plt.plot(m_list, model(m_list, *popt), label="fit", color="tab:red")
plt.plot(m_list, y_true, "--", label="true", color="tab:green")
plt.xlabel("m (sequence length)")
plt.ylabel("survival probability")
plt.legend()
plt.show()


In [ ]:
from typing import List, Tuple, Sequence
import cirq

all_c = cirq.SingleQubitCliffordGate.all_single_qubit_cliffords
print(len(all_c))   # 应该是 24
print(all_c[:3])    # 看前三个
for i in range(24):
    print(cirq.unitary(all_c[i]))
    i = i + 1


In [ ]:
import numpy as np

rng = np.random.RandomState()

rng.choice(24, 5)

In [ ]:
import cirq

q1, q2, q3, q4 = cirq.LineQubit.range(4)

qubits = [q1, q2, q3, q4]
for (qubit, i) in zip(qubits, range(len(qubits))):
    print(f"qubit {i}: {qubit}")

In [ ]:
import cirq
import numpy as np
from cirq.noise import _sample_moment

q1, q2, q3,q4,q5 = cirq.LineQubit.range(5)

qubits = [q1, q2, q3, q4, q5]
depth = 5
seed = 20251009

rb_moment = _sample_moment(qubits)

circuit = cirq.Circuit()

circuit.append(rb_moment)

print(circuit)

In [ ]:
import cirq
import numpy as np
from cirq.noise import build_rb1_circuit

q1, q2, q3, q4 = cirq.LineQubit.range(4)

qubits = [q1, q2, q3]
depth = 5
seed = 20251009
rng = np.random.RandomState(seed)
rb_circuit = build_rb1_circuit(qubits, depth, measure = False)

print(rb_circuit)

In [ ]:
from experiments import run_exp2_experiment, save_results_exp2, make_run_dir, load_config

# 1) 读取 YAML
cfg = load_config("exp2_rb.yaml")

# 2) 跑实验（返回结果 + 元数据）
results = run_exp2_experiment(cfg)

run_dir = make_run_dir(exp_name="experiment2")

# 3) 保存结果（带 header）

save_results_exp2(cfg, results, run_dir)



In [ ]:
from experiments import (
    run_interleaved_rb_experiment,
    save_results_interleaved_rb,
    make_run_dir,
    load_config,
)

# === 1. 读取 YAML 配置 ===
cfg = load_config("exp2_interleaved_rb.yaml")

# === 2. 执行实验（包含参考 + 每个门的 interleaved）===
results = run_interleaved_rb_experiment(cfg)

# === 3. 创建运行输出目录（默认存入 results/exp2_interleaved_rb/下）===
run_dir = make_run_dir(exp_name="exp2_interleaved_rb")

# === 4. 保存结果文本文件（results.txt）===
save_results_interleaved_rb(cfg, results, run_dir)


#### test ctx again(solved)

In [ ]:
import cirq
from cirq.noise import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, CompositeNoiseModel, SegmentedFluxNoiseModel, assign_timed_circuit_context, sample_flux_noise_segments, fidelity, trace_distance

# 定义三比特
q0, q1, q2 = cirq.LineQubit.range(3)
circuit_no_measure = cirq.Circuit(
        cirq.H(q0),
        cirq.CX(q0, q1),
        cirq.CX(q1, q2),
        cirq.H(q1),
        cirq.CX(q0, q1),
    )

print(circuit_no_measure)

context = assign_timed_circuit_context(circuit_no_measure, tau_c=60)

context.print_timing_map()


In [ ]:
import cirq
from cirq.noise.models import FluxNoiseContext, SegmentedFluxNoiseModel,  sample_flux_noise_segments
from cirq.noise.utils import assign_timed_circuit_context

q0, q1, q2 = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.H(q1),
    cirq.CX(q0, q1),
)

timing_context = assign_timed_circuit_context(circuit, tau_c=60)

delta_phis = sample_flux_noise_segments([q0, q1, q2], 4)

noise = SegmentedFluxNoiseModel(timing_context.timing_map, delta_phis)

simulator = cirq.DensityMatrixSimulator(noise)

result = simulator.run(circuit, repetitions=1000)

#### test easy noise model

In [ ]:
import cirq
from cirq.noise.utils.easy_gatelevel_model import identify_gate_name, estimate_circuit_fidelity

q0, q1, q2 = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CZ(q0, q1),
    cirq.CZ(q1, q2),
)

gate_errors = {
    'H': 0.001,
    'CZ': 0.01,
}

fid = estimate_circuit_fidelity(circuit, gate_errors)

print(circuit)
print(f"Estimated Circuit Fidelity: {fid:.6f}")



#### average behavior

Mean fidelities (to ideal state):
    m=20   F̄=0.896513
    m=40   F̄=0.803298
    m=60   F̄=0.687619
    m=80   F̄=0.613164
    m=100  F̄=0.542766
    m=120  F̄=0.502057
    m=140  F̄=0.436829
    m=160  F̄=0.400454
    m=180  F̄=0.378097
    m=200  F̄=0.339564

[Results: Ablations]
Noise component   EPC(-i)         Δr              Contribution(%) 
------------------------------------------------------------------
use_idle          0.004973        0.001168        19.11           
use_ry            0.004302        0.001839        30.08           
use_photon_decay  0.004504        0.001637        26.77           
use_segmented_flux0.004671        0.001470        24.05           
------------------------------------------------------------------
Sum Δr = 6.115482e-03

Mean fidelities (to ideal state):
    m=20   F̄=0.903222
    m=40   F̄=0.787166
    m=60   F̄=0.691521
    m=80   F̄=0.619571
    m=100  F̄=0.551586
    m=120  F̄=0.501074
    m=140  F̄=0.446733
    m=160  F̄=0.407804 
    m=180  F̄=0.364419
    m=200  F̄=0.333457

[Results: Ablations]
Noise component   EPC(-i)         Δr              Contribution(%) 
------------------------------------------------------------------
use_idle          0.004654        0.001547        25.16           
use_ry            0.003780        0.002420        39.37           
use_photon_decay  0.004020        0.002180        35.47           
------------------------------------------------------------------
Sum Δr = 6.147354e-03

In [ ]:
import cirq
import numpy as np
from cirq.noise import PhotonDecayNoiseModel, RyGateNoiseModel, CompositeNoiseModel, IdleNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.metrics import fidelity


q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.Z(q0),
    cirq.ry(np.pi/2)(q2),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
    # cirq.measure(q0, q1, q2, key='m')
)

timinged_context = assign_timed_circuit_context(circuit, tau_c=60)
max_seg = max(info.segment_id for infos in timinged_context.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(sorted(circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q))), max_seg)

model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel(kappa = 4e6,timed_context = timinged_context)
model3 = RyGateNoiseModel(z_policy = 'virtual')
model4 = SegmentedFluxNoiseModel(timing_map = timinged_context.timing_map, delta_phis= delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])

noisy_circuit = circuit.with_noise(model1)

print(f"Original Circuit:\n{circuit}")

print(f"\nNoisy Circuit:\n{noisy_circuit}")

sim_ideal = cirq.DensityMatrixSimulator()
rho_ideal = sim_ideal.simulate(circuit).final_density_matrix

sim_noisy = cirq.DensityMatrixSimulator()
rho_noisy = sim_noisy.simulate(noisy_circuit).final_density_matrix

fid = fidelity(rho_ideal, rho_noisy)
print(f"\nFidelity between ideal and noisy final states: {fid:.6f}")


In [ ]:
import supermarq

c = supermarq.ghz.GHZ(5).cirq_circuit()
print(c)


In [ ]:
import cirq
from supermarq.benchmarks import BitCode
from cirq.noise.utils.compilation_scheme import all_in_one_compile

bc = BitCode(num_data_qubits=4, num_rounds=100, bit_state=[0, 1, 0, 1])
c_unit: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit

c_unit = [moment for moment in c_unit if all(not (isinstance(op.gate, cirq.MeasurementGate) or isinstance(op.gate, cirq.ResetChannel)) for op in moment.operations)]

c = cirq.Circuit(c_unit * 1)

cp = all_in_one_compile(c)
print(cp)

from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context

ctx = assign_timed_circuit_context(cp)
ctx.print_timing_map()


In [ ]:
import cirq
from supermarq.benchmarks import HamiltonianSimulation

bc = HamiltonianSimulation(num_qubits=6, time_step=1, total_time = 4)
c: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit
# print(c[:-1])
print(all_in_one_compile(c[:-1]))

In [ ]:
import cirq
from supermarq.benchmarks import MerminBell

bc = MerminBell(num_qubits=8)
c: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit
c = cirq.Circuit([moment for moment in c if all(not (isinstance(op.gate, cirq.MeasurementGate) or isinstance(op.gate, cirq.ResetChannel)) for op in moment.operations)])
print(all_in_one_compile(c))


In [ ]:
import cirq
from supermarq.benchmarks import PhaseCode

bc = PhaseCode(num_data_qubits=3, num_rounds=1, phase_state=[0,0,0])
c: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit
print(c)

In [ ]:
import cirq
from supermarq.benchmarks import QAOAFermionicSwapProxy

bc = QAOAFermionicSwapProxy(num_qubits=4)
c: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit
print(c)

In [ ]:
import cirq
import numpy as np

from cirq.transformers import (
    optimize_for_target_gateset,
    merge_single_qubit_gates_to_phased_x_and_z,
    eject_z,
    eject_phased_paulis,
    expand_composite,
    drop_negligible_operations,
    drop_empty_moments,
    gauge_compiling,
)
from cirq.transformers.gauge_compiling.cz_gauge import CZGaugeTransformer
from cirq.transformers import target_gatesets as tg

def _x_to_zyz(op: cirq.Operation, _moment_index: int):
    g = getattr(op, "gate", None)
    if isinstance(g, cirq.XPowGate):
        t = g.exponent
        q = op.qubits[0]
        return [
            cirq.ZPowGate(exponent=-0.5).on(q),
            cirq.YPowGate(exponent=t).on(q),
            cirq.ZPowGate(exponent=+0.5).on(q),
        ]
    return op


def compile_to_Rz_Ry_CZ(c: cirq.Circuit, seed: int | None = 1234) -> cirq.Circuit:
    # 1) 任意两比特 → CZ + 单比特（官方 gateset 编译）
    c = optimize_for_target_gateset(c, gateset=tg.CZTargetGateset())

    # 2) 对每个 CZ 插入 pre/post 的本地 Clifford（随机可复现）
    # c = CZGaugeTransformer(c, prng=np.random.default_rng(seed))  # deep=False 是默认要求

    # 2) 单比特门规约：合并→PhasedXZ，推出虚拟Z，去掉相位化保利
    c = merge_single_qubit_gates_to_phased_x_and_z(c)
    c = eject_z(c)                  # 尽量把 Z 变成虚拟旋转/与两比特门对齐
    c = eject_phased_paulis(c)      # PhasedX → Z·X·Z 形式

    # 3) 展开复合门（得到 ZPow / XPow / …）
    c = expand_composite(c)

    # 4) 最后一小步自定义：XPow → Z(-1/2) · YPow · Z(+1/2)
    c = cirq.map_operations_and_unroll(c, _x_to_zyz)
    
    # 5) 清理
    c = drop_negligible_operations(c)
    c = drop_empty_moments(c)

    # 断言只剩 {Rz, Ry, CZ, M}
    for op in c.all_operations():
        if not isinstance(op.gate, (cirq.ZPowGate, cirq.YPowGate, cirq.CZPowGate, cirq.MeasurementGate)):
            raise AssertionError(f"Unexpected gate after compile: {op!r}")
    return c

q0, q1, q2 = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
)

print(f"Original Circuit:\n{circuit}")

compiled_circuit = compile_to_Rz_Ry_CZ(circuit)

print(f"\nCompiled Circuit (Rz, Ry, CZ):\n{compiled_circuit}")

print(compiled_circuit.moments)



In [ ]:
import cirq
import numpy as np
from cirq.transformers import (
    optimize_for_target_gateset,
    # 某些版本叫 merge_single_qubit_gates_into_phased_x_z：
    merge_single_qubit_gates_to_phased_x_and_z as merge_1q_into_phxz,
    eject_z, eject_phased_paulis, expand_composite,
    drop_negligible_operations, drop_empty_moments,
)
from cirq.transformers import target_gatesets as tg
from cirq.transformers.gauge_compiling.cz_gauge import CZGaugeTransformer

def _x_to_zyz(op: cirq.Operation, _moment_index: int):
    g = getattr(op, "gate", None)
    if isinstance(g, cirq.XPowGate):
        t = g.exponent
        q = op.qubits[0]
        return [
            cirq.ZPowGate(exponent=-0.5).on(q),  # Rz(-π/2)
            cirq.YPowGate(exponent=t).on(q),     # Ry(π t)
            cirq.ZPowGate(exponent=+0.5).on(q),  # Rz(+π/2)
        ]
    return op

def _post_cleanup_rz_ry_cz(c: cirq.Circuit, *, atol=1e-9, max_iters=6) -> cirq.Circuit:
    """反复合并/推出/展开，直到 op 数不再下降。"""
    def opcount(cc): return sum(1 for _ in cc.all_operations())
    prev = None
    for _ in range(max_iters):
        before = opcount(c)
        # 合并相邻 1Q 到 PhasedXZ（可把 S·S^-1、T·T^-1 等直接吃掉）
        c = merge_1q_into_phxz(c)
        # 尽量把 Z 当成虚拟旋转往左右推出（Z 与 CZ 交换，能被吸收到相邻1Q里）
        c = eject_z(c, atol=atol)
        c = eject_phased_paulis(c, atol=atol)
        # 展开复合门（把 PhasedX/Z 再摊成原子门，便于下一步替换 X→Z–Y–Z）
        c = expand_composite(c)
        # 把 X 轴旋转等价替换为 Z–Y–Z，再立刻展开 map 产生的 CircuitOp
        try:
            from cirq.transformers import map_operations_and_unroll
            c = map_operations_and_unroll(c, _x_to_zyz)
        except Exception:
            c = cirq.map_operations(c, _x_to_zyz)
            c = expand_composite(c)
        # 丢掉极小旋转/空 moment
        c = drop_negligible_operations(c, atol=atol)
        c = drop_empty_moments(c)

        after = opcount(c)
        if prev is not None and after >= prev:  # 没继续变小就停
            break
        prev = after
        if after == before:
            break
    return c

def compile_to_Rz_Ry_CZ(c: cirq.Circuit, seed: int | None = 1234, *, atol=1e-9) -> cirq.Circuit:
    # 1) 任意两比特 → CZ + 1Q
    c = optimize_for_target_gateset(c, gateset=tg.CZTargetGateset())
    # 2) 可选：对每个 CZ 做 gauge（本地 Clifford 前后吸收，后续会被合并/推出吃掉）
    c = CZGaugeTransformer(c, prng=np.random.default_rng(seed))
    # 3) 第一轮规范
    c = _post_cleanup_rz_ry_cz(c, atol=atol)
    # 4) 断言只剩 Rz/Ry/CZ/M
    for op in c.all_operations():
        if not isinstance(op.gate, (cirq.ZPowGate, cirq.YPowGate, cirq.CZPowGate, cirq.MeasurementGate)):
            raise AssertionError(f"Unexpected gate after compile: {op!r}")
    return c

q0, q1, q2 = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
)

print(f"Original Circuit:\n{circuit}")

compiled_circuit = compile_to_Rz_Ry_CZ(circuit)

print(f"\nCompiled Circuit (Rz, Ry, CZ):\n{compiled_circuit}")

print(circuit.moments)


In [ ]:
import numpy as np
import cirq
from cirq.noise.utils.compilation_scheme import compile_to_Rz_Ry_CZ


q0, q1, q2 = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.CX(q0, q1),
    cirq.CX(q1, q2),
)

print(f"Original Circuit:\n{circuit}")

compiled_circuit = compile_to_Rz_Ry_CZ(circuit)

print(f"\nCompiled Circuit (Rz, Ry, CZ):\n{compiled_circuit}")

# print(compiled_circuit.moments)

# ----------------------
from cirq.noise import PhotonDecayNoiseModel, RyGateNoiseModel, CompositeNoiseModel, IdleNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context

timinged_context = assign_timed_circuit_context(compiled_circuit, tau_c=60)
max_seg = max(info.segment_id for infos in timinged_context.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(sorted(compiled_circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q))), max_seg)

model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel(timed_context = timinged_context)
model3 = RyGateNoiseModel()
model4 = SegmentedFluxNoiseModel(timing_map = timinged_context.timing_map, delta_phis= delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])

noisy_circuit = compiled_circuit.with_noise(model1)

print(noisy_circuit)

#### benchmark test

In [ ]:
import cirq
import numpy as np

from cirq.noise.utils.compilation_scheme import compile_to_Rz_Ry_CZ
from cirq.noise.models import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.composite_noise_model import CompositeNoiseModel

from supermarq.benchmarks import BitCode

bc = BitCode(num_data_qubits=3, num_rounds=1, bit_state=[0,0,0])
circuit: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit

compiled_circuit = compile_to_Rz_Ry_CZ(circuit)

timinged_context = assign_timed_circuit_context(compiled_circuit, tau_c=60)
max_seg = max(info.segment_id for infos in timinged_context.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(sorted(compiled_circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q))), max_seg)

model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel()
model3 = RyGateNoiseModel()
model4 = SegmentedFluxNoiseModel(timing_map = timinged_context.timing_map, delta_phis= delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])

noisy_circuit = compiled_circuit.with_noise(composite_model) 

print(f"original circuit:\n{circuit}\n")

print(f"compiled circuit:\n{compiled_circuit}\n")

print(f"noisy circuit:\n{noisy_circuit}\n")

In [ ]:
from experiments import (
    run_exp3_experiment,
    make_run_dir,
    load_config,
    save_results_exp3,
)

# === 1. 读取 YAML 配置 ===
cfg = load_config("exp3.yaml")

run_dir = make_run_dir(exp_name="experiment3")

# === 2. 执行实验（包含参考 + 每个门的 interleaved）===
results = run_exp3_experiment(cfg)

save_results_exp3(cfg, results, out_path = run_dir)


In [ ]:
import cirq
from supermarq.benchmarks import BitCode, MerminBell

from cirq.noise.models import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.composite_noise_model import CompositeNoiseModel

from cirq.noise.utils.metrics import fidelity, trace_distance

from cirq.noise.utils.compilation_scheme import compile_to_Rz_Ry_CZ

from cirq.noise.utils.easy_gatelevel_model import identify_gate_name, estimate_circuit_fidelity

def remove_measurements(circuit: cirq.Circuit) -> cirq.Circuit:
    return cirq.Circuit(
        op for op in circuit.all_operations()
        if not isinstance(op.gate, cirq.MeasurementGate) and not isinstance(op.gate, cirq.ResetChannel)
    )

bc = BitCode(num_data_qubits=5, num_rounds=4, bit_state=[0,0,0,0,0])
c: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit
# bc = MerminBell(num_qubits=8)
# c: cirq.Circuit = bc.circuit()  # 新版直接 .circuit

c = compile_to_Rz_Ry_CZ(c)
c = remove_measurements(c)

ctx = assign_timed_circuit_context(c, t1 = 20, t2 = 40, tau_c=60)
max_seg = max(info.segment_id for infos in ctx.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(sorted(c.all_qubits(), key=lambda q: getattr(q, "x", str(q))), max_seg)
model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel(kappa=6e6)
model3 = RyGateNoiseModel()
model4 = SegmentedFluxNoiseModel(timing_map = ctx.timing_map, delta_phis= delta_phis)
composite_model = CompositeNoiseModel([model1, model2, model3, model4])

sim_ideal = cirq.DensityMatrixSimulator()
rho_ideal = sim_ideal.simulate(c).final_density_matrix

sim_noisy = cirq.DensityMatrixSimulator(noise = composite_model)
rho_noisy = sim_noisy.simulate(c).final_density_matrix

fid = fidelity(rho_ideal, rho_noisy)

print(c)

print("Fidelity between ideal and noisy circuit:", fid)

gate_errors = {
    'H': 0.002288,
    'CZ': 0.004961,
    'Y90+': 0.003701,
    'Y90-': 0.003687,
    'H': 0.002288,
    'IDLE': 0.001713,
}


print("Estimated Circuit Fidelity via gate-level model:", estimate_circuit_fidelity(c, gate_errors))

print(f"noisy circuit:\n{c.with_noise(composite_model)}\n")

In [ ]:
import cirq
from supermarq.benchmarks import BitCode, GHZ
from cirq.noise.utils.easy_gatelevel_model import estimate_circuit_fidelity
from cirq.noise.utils.compilation_scheme import compile_to_Rz_Ry_CZ


bc = BitCode(num_data_qubits=3, num_rounds=3, bit_state=[0,0,0])
c: cirq.Circuit = bc.circuit()  # 新版直接 .circuit() 返回 Cirq Circuit

gate_errors = {
    'H': 0.002288,
    'CZ': 0.004961,
    'Measure': 0.02,
    'Reset': 0.02,
    'Y90+': 0.003701,
    'Y90-': 0.003687,
    'H': 0.002288,
    'IDLE': 0.001713
}

q0, q1, q2, q3 = cirq.LineQubit.range(4)

circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CZ(q0, q1),
    cirq.CZ(q1, q2),
    cirq.CZ(q2, q3),
    cirq.measure(q0, q1, q2, q3, key='m'),
    cirq.H(q0),
    cirq.CZ(q0, q1),
    cirq.CZ(q1, q2),
    cirq.CZ(q2, q3),
    cirq.YPowGate(exponent=0.5).on(q0),
    cirq.YPowGate(exponent=-0.5).on(q1),
)



bc1 = GHZ(num_qubits=13)
c1 = bc1.circuit()
print(c1)

c1 = compile_to_Rz_Ry_CZ(c1)
print(c1)

fid = estimate_circuit_fidelity(c1, gate_errors)
print(f"Estimated Circuit Fidelity: {fid:.6f}")



#### ctx with noise

In [ ]:
import cirq
from cirq.noise import PhotonDecayNoiseModel, RyGateNoiseModel, CompositeNoiseModel, IdleNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.compilation_scheme import compile_to_Rz_Ry_CZ


# 定义三比特
q0, q1, q2, q3 = cirq.LineQubit.range(4)
circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.H(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)

print(f"Original Circuit:\n{circuit}")

context = assign_timed_circuit_context(circuit, tau_c=60, treat_z_as_virtual = False)

context.print_timing_map()

max_seg = max(info.segment_id for infos in context.timing_map.values() for info in infos)
all_qubits = sorted(circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q)))
delta_phis = sample_flux_noise_segments(all_qubits, max_seg)

model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel(timed_context = context)
model3 = RyGateNoiseModel()
model4 = SegmentedFluxNoiseModel(timing_map = context.timing_map, delta_phis= delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])

compiled_circuit = compile_to_Rz_Ry_CZ(circuit)

print(f"\nCompiled Circuit:\n{compiled_circuit}")

new_context = assign_timed_circuit_context(compiled_circuit, tau_c=60)

new_context.print_timing_map()

noisy_circuit = compiled_circuit.with_noise(model1)

print(f"\nNoisy Circuit:\n{noisy_circuit}")


In [ ]:
import cirq
from cirq.noise import PhotonDecayNoiseModel, RyGateNoiseModel, CompositeNoiseModel, IdleNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.compilation_scheme import compile_to_Rz_Ry_CZ

q0, q1, q2, q3 = cirq.LineQubit.range(4)

p1 = cirq.CircuitOperation(cirq.FrozenCircuit(
    [cirq.T(q1),
    cirq.YPowGate(exponent=0.5).on(q1),]
))

circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.T(q1),
    cirq.YPowGate(exponent=0.5).on(q1),
    cirq.H(q0),
    cirq.H(q2),
    cirq.YPowGate(exponent=-0.5).on(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)

print(f"Circuit:\n{circuit}")

context = assign_timed_circuit_context(circuit, tau_c=60, treat_z_as_virtual = True)

context.print_timing_map()

# circuit.moments[0] = cirq.Moment([o for o in circuit.moments[0] if o != circuit.moments[0].operations[1]])

# operations = list(circuit.moments[0])
# operations.append(p1)
# circuit.moments[0] = cirq.Moment(operations)
print(f"circuit:\n{circuit}")

#### base compilation test

In [ ]:
import cirq

q0, q1, q2, q3 = cirq.LineQubit.range(4)
circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.H(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)

print(f"Original Circuit:\n{circuit}")

In [ ]:
import cirq
from cirq.noise.utils.circuit_factory import build_QSIH_circuits

circuit_sym, circuit, qubits = build_QSIH_circuits(n_qubits = 4, n_layers = 2, theta_value = 0.05)

print(f"Original Circuit:\n{circuit}")

In [ ]:
import cirq
from cirq.noise.utils.metrics import fidelity

q0, q1 = cirq.LineQubit.range(2)
circuit1 = cirq.Circuit(
    cirq.CX(cirq.LineQubit(0), cirq.LineQubit(1)),
)

circuit2 = cirq.Circuit(
    cirq.YPowGate(exponent=0.5).on(cirq.LineQubit(1)),
    cirq.CZ(cirq.LineQubit(0), cirq.LineQubit(1)),
    cirq.YPowGate(exponent=-0.5).on(cirq.LineQubit(1)),
)

print(f"Circuit 1:\n{circuit1}\nCircuit 2:\n{circuit2}")

sim = cirq.DensityMatrixSimulator()
rho1 = sim.simulate(circuit1).final_density_matrix
rho2 = sim.simulate(circuit2).final_density_matrix

fid = fidelity(rho1, rho2)

print(f"\nfidelity between circuit1 and circuit2: {fid:.6f}")


In [7]:
import cirq
import numpy as np


from cirq.noise.utils.metrics import fidelity
from cirq.transformers import optimize_for_target_gateset
from cirq.noise.utils.compilation_scheme import CZ_Y_Z_gateset, merge_zpow_weak, merge_zpow_subcircuit, asapize_circuit
from cirq.noise.utils.circuit_factory import build_QSIH_circuits

gset = CZ_Y_Z_gateset()

q0, q1, q2, q3 = cirq.LineQubit.range(4)
circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.H(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)
# circuit_sym, circuit, _ = build_QSIH_circuits(n_qubits = 6, n_layers = 20, theta_value = 0.05)
# from supermarq.benchmarks import HamiltonianSimulation
# bc = HamiltonianSimulation(num_qubits=6, time_step=1, total_time = 138)
# circuit: cirq.Circuit = bc.circuit()[:-1]  # 新版直接 .circuit() 返回 Cirq Circuit
sim = cirq.DensityMatrixSimulator()
rho0 = sim.simulate(circuit).final_density_matrix
# print(f"original circuit:\n{circuit}")

circuit_p1 = optimize_for_target_gateset(circuit, gateset=gset)
rho1 = sim.simulate(circuit_p1).final_density_matrix
# print(f"circuit_p1:\n{circuit_p1}\nfidelity with original:{fidelity(rho0, rho1)}")

circuit_p2 = merge_zpow_weak(circuit_p1)
rho2 = sim.simulate(circuit_p2).final_density_matrix
# print(f"circuit_p2:\n{circuit_p2}\nfidelity with original:{fidelity(rho0, rho2)}")

circuit_p3 = merge_zpow_subcircuit(circuit_p2)
rho3 = sim.simulate(circuit_p3).final_density_matrix
# print(f"circuit_p3:\n{circuit_p3}\nfidelity with original:{fidelity(rho0, rho3)}")

circuit_p4 = asapize_circuit(circuit_p3)
rho4 = sim.simulate(circuit_p4).final_density_matrix
print(f"circuit_p4:\n{circuit_p4}\nfidelity with original:{fidelity(rho0, rho4)}")
# print(f"fidelity with self:{fidelity(rho0, rho0)}")
# print(f"fidelity1 with self:{fidelity1(rho0, rho0)}")
# print(f"tr rho0:{np.trace(rho0)}\n tr rho4:{np.trace(rho4)}")

# ----------------------

from cirq.noise.models import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.composite_noise_model import CompositeNoiseModel

timinged_context = assign_timed_circuit_context(circuit_p4, 20, 40, tau_c=20)
max_seg = max(info.segment_id for infos in timinged_context.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(sorted(circuit_p4.all_qubits(), key=lambda q: getattr(q, "x", str(q))), max_seg)


model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel(ctx = timinged_context)
model3 = RyGateNoiseModel()
model4 = SegmentedFluxNoiseModel(timing_map = timinged_context.timing_map, delta_phis = delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])


circuit_p4:
                                     [ 0: ───@─── ]
0: ───[ 0: ───Z───Y^0.5─── ]─────────[       │    ]───────[ 0: ───Z^0.324───Y───Z^-0.676─── ]─────────────────────────────────────────────────────
                                     [ 1: ───@─── ]
                                     │
                                     │                                                          [ 1: ───@───Z─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5─── ]───#2───────────────────[ 1: ───Y^0.5─── ]────────────────────[       │        ]────────────────────────────────
                                                                                                [ 2: ───@─────── ]
                                                                                                │
      [ 2: ───@─── ]                                                                            │
2: ───[       │    ]─────────────────[ 2: ───Y^0.5─── ]─────────────────────────────────────────#2───────────────────[ 2: ───

In [8]:
model2.reset()
model4.reset()

noisy_circuit = circuit_p4.with_noise(model4)
print(f"noisy circuit(tau_c = 20 ns):\n{noisy_circuit}")

noisy circuit(tau_c = 20 ns):
                                                       [ 0: ───@───eD(-1.37e-02)───eE(6.73e-02)─── ]
0: ───[ 0: ───Z───Y^0.5───eD(-2.95e-02)─── ]───────────[       │                   │               ]───[ 0: ───Z^0.324───Y───Z^-0.676───eD(5.37e-02)─── ]─────────────────────────────────────────────────────────────────────────────────────────────────────
                                                       [ 1: ───@───────────────────eE(6.73e-02)─── ]
                                                       │
                                                       │                                                                                                    [ 1: ───@───Z───eD(-1.54e-02)───eE(-5.50e-02)─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───eD(-2.19e-02)─── ]─────#2──────────────────────────────────────────────[ 1: ───Y^0.5───eD(3.10e-02)─── ]────────────────────[       │                       │                ]───────────────────────────────────────────

In [2]:
timinged_context.print_timing_map()

timing for operations (sorted by segment and qubit):

 seg |    t (ns) |  dur (ns) |     qubits |  occ |     gate
-----------------------------------------------------------
   0 |       0.0 |      20.0 |        (0) |    0 |  Y**0.75
   0 |       0.0 |      20.0 |        (1) |    0 |  Y**0.25
   0 |       0.0 |      20.0 |        (2) |    0 |  Y**0.25
   0 |       0.0 |      20.0 |        (3) |    0 |  Y**0.25
   0 |       0.0 |      20.0 |        (4) |    0 |  Y**0.25
   0 |       0.0 |      20.0 |        (5) |    0 | Y**0.7499999999999996
   0 |      20.0 |      40.0 |      (0,1) |    0 |       CZ
   1 |      60.0 |      20.0 |        (0) |    0 | Y**0.7493178646510268
   1 |      60.0 |      20.0 |        (1) |    0 |        Y
   1 |      80.0 |      40.0 |      (1,2) |    0 |       CZ
   2 |     120.0 |      20.0 |        (1) |    0 | Y**0.749317864651027
   2 |     120.0 |      20.0 |        (2) |    0 |        Y
   2 |     140.0 |      40.0 |      (0,1) |    1 |       CZ
   2 |  

In [ ]:
from cirq.noise.utils.circuit_timing_cal import compute_timing_summary

timing_summary = compute_timing_summary(
        circuit_p4,
        20,
        40,
    )

print(timing_summary)
sum(timing_summary['per_moment_durations_ns'])

In [ ]:
# for i, moment in enumerate(circuit_p4.moments):
#     print(f"Moment {i}:\n")
#     for op in moment.operations:
#         print(f"  {op}")
model2.reset()
model4.reset()
noisy_circuit = circuit_p4.with_noise(model1)
print(f"\nNoisy Circuit:\n{noisy_circuit}")

rho_noisy = sim.simulate(noisy_circuit).final_density_matrix
fid_noisy = fidelity(rho0, rho_noisy)
print(f"\nFidelity1 between original and noisy circuit: {fid_noisy:.6f}")

#### compilation scheme test

In [ ]:
import cirq
import numpy as np
from cirq.noise.utils.compilation_scheme import merge_zpow_subcircuit

circuit_new = merge_zpow_subcircuit(compiled_circuit)

print(f"\nMerged Circuit:\n{circuit_new}")

# print(circuit_new.moments)

In [ ]:
import cirq

from cirq.noise.utils.composite_noise_model import CompositeNoiseModel
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.compilation_scheme import compile_to_Rz_Ry_CZ, merge_zpow_subcircuit, asapize_circuit

# q0, q1, q2, q3 = cirq.LineQubit.range(4)
# circuit = cirq.Circuit(
#     cirq.S(q0),
#     cirq.S(q0),
#     cirq.T(q1),
#     cirq.H(q0),
#     cirq.H(q3),
#     cirq.CX(q0, q1),
#     cirq.CX(q2, q3),
#     cirq.CX(q1, q2),
# )

import sympy, numpy as np
from typing import Tuple, List

def _U_block_QSIH(q0, q1, tx, ty, tz):
    return [
        cirq.CNOT(q0, q1),
        cirq.rz(tz)(q0),
        cirq.H(q1),
        cirq.rz(tx + np.pi/2)(q1),
        cirq.CNOT(q0, q1),
        cirq.rz(-ty)(q0),
        cirq.H(q1),
        cirq.CNOT(q0, q1),
        cirq.rx(np.pi/2)(q0),
        cirq.rx(-np.pi/2)(q1),
    ]

def _add_layer_QSIH(c: cirq.Circuit, qs: List[cirq.Qid], thetas, even=True):
    start = 0 if even else 1
    for i in range(start, len(qs)-1, 2):
        tx, ty, tz = thetas[i]
        c.append(_U_block_QSIH(qs[i], qs[i+1], tx, ty, tz))

def build_QSIH_circuits(n_qubits: int, n_layers: int, theta_value: float
                       ) -> Tuple[cirq.Circuit, cirq.Circuit, List[cirq.Qid]]:
    """
    构建具有自定义参数的 QSIH 电路。

    参数：
        n_qubits: 量子比特数量（至少 2）
        n_layers: QSIH 层数（至少 1）
        theta_value: 用于数值电路的构建参数

    返回:
        Tuple[cirq.Circuit, cirq.Circuit, List[cirq.Qid]]:
            符号电路、数值电路、量子比特列表
    """
    qs = cirq.LineQubit.range(n_qubits)
    c_sym, c_num = cirq.Circuit(), cirq.Circuit()

    def syms(l, i):
        return (sympy.Symbol(f"theta_x_{l}_{i}"),
                sympy.Symbol(f"theta_y_{l}_{i}"),
                sympy.Symbol(f"theta_z_{l}_{i}"))

    for l in range(n_layers):
        sym_layer = [syms(l, i) for i in range(n_qubits-1)]
        num_layer = [(theta_value, theta_value, theta_value)]*(n_qubits-1)
        _add_layer_QSIH(c_sym, qs, sym_layer, even=(l%2==0))
        _add_layer_QSIH(c_num, qs, num_layer, even=(l%2==0))

    return c_sym, c_num, qs

circuit_sym, circuit, qubits = build_QSIH_circuits(n_qubits=4, n_layers=2, theta_value=0.05)

print(f"Original Circuit:\n{circuit}")

compiled_ins_circuit = compile_to_Rz_Ry_CZ(circuit)

print(f"\nCompiled ins Circuit:\n{compiled_ins_circuit}")

compiled_circuit = merge_zpow_subcircuit(compiled_ins_circuit)

print(f"\nMerged Circuit:\n{compiled_circuit}")

asap_circuit = asapize_circuit(compiled_circuit)

print(f"\nASAPized Circuit:\n{asap_circuit}")

context = assign_timed_circuit_context(asap_circuit, tau_c=100)

context.print_timing_map()

for i, moment in enumerate(asap_circuit.moments):
    print(f"moment[{i}]:")
    for j, op in enumerate(moment.operations):
        print(f"op[{j}]:\n{op}")
    print("\n")

In [ ]:
from cirq.noise.models import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments

max_seg = max(info.segment_id for infos in context.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(sorted(asap_circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q))), max_seg)

model1 = IdleNoiseModel()
model2 = PhotonDecayNoiseModel(timed_context = context)
model3 = RyGateNoiseModel(z_policy = 'virtual')
model4 = SegmentedFluxNoiseModel(timing_map = context.timing_map, delta_phis= delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])

noisy_circuit = asap_circuit.with_noise(composite_model)

print(f"\nNoisy Circuit:\n{noisy_circuit}")

for i, moment in enumerate(noisy_circuit.moments):
    print(f"moment[{i}]:")
    for j, op in enumerate(moment.operations):
        print(f"op[{j}]:\n{op}")
    print("\n")

sim_ideal = cirq.DensityMatrixSimulator()
rho_ideal = sim_ideal.simulate(circuit).final_density_matrix

sim_noisy = cirq.DensityMatrixSimulator()
rho_noisy = sim_noisy.simulate(compiled_ins_circuit).final_density_matrix

from cirq.noise.utils.metrics import fidelity
fid = fidelity(rho_ideal, rho_noisy)
print(f"\nFidelity between ideal and noisy final states: {fid:.6f}")



In [ ]:
import cirq
from cirq.noise import PhotonDecayNoiseModel, RyGateNoiseModel, CompositeNoiseModel, IdleNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context

# 定义三比特
q0, q1, q2, q3 = cirq.LineQubit.range(4)
before_compile_circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.YPowGate(exponent=0.5).on(q1),
    cirq.H(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)

print(f"Original Circuit:\n{before_compile_circuit}")

context = assign_timed_circuit_context(before_compile_circuit, tau_c=60, treat_z_as_virtual = False)

context.print_timing_map()

max_seg = max(info.segment_id for infos in context.timing_map.values() for info in infos)
all_qubits = sorted(before_compile_circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q)))
delta_phis = sample_flux_noise_segments(all_qubits, max_seg)

model1 = IdleNoiseModel(treat_z_as_virtual = False)
model2 = PhotonDecayNoiseModel(ctx = context)
model3 = RyGateNoiseModel()
model4 = SegmentedFluxNoiseModel(timing_map = context.timing_map, delta_phis= delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])

noisy_circuit = before_compile_circuit.with_noise(model1)

print(f"\nNoisy Circuit:\n{noisy_circuit}")

In [ ]:
from cirq.noise.utils.circuit_timing_cal import compute_timing_summary

timing_summary = compute_timing_summary(noisy_circuit, t1 = 20, t2 = 40)
print(f"\nTiming Summary:{timing_summary}\n")

#### test average again

In [ ]:
import cirq

from cirq.noise.models.average_flux_noise_model import AverageFluxNoiseModel

q0, q1, q2, q3 = cirq.LineQubit.range(4)
circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.H(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)

print(f"Original Circuit:\n{circuit}")

noise_model = AverageFluxNoiseModel(0.05)

noisy_circuit = circuit.with_noise(noise_model)
print(f"\nNoisy Circuit:\n{noisy_circuit}")

In [ ]:
import numpy as np


rng = np.random.default_rng()

for _ in range(10):
    print(rng.normal(0, 0.05))

#### [re] exp1

In [2]:

from experiments.scripts.experiment1_fid import run_exp1_experiment, save_results_exp1
from experiments.utils import make_run_dir, load_config

# 1) 读取 YAML
cfg = load_config("exp1_fid.yaml")

# 2) 跑实验（返回结果 + 元数据）
results, meta = run_exp1_experiment(cfg)

run_dir = make_run_dir(exp_name="experiment1")
# 3) 保存结果（带 header）
paths = save_results_exp1(
    results, meta, run_dir,                 # 有就传
    results_filename="results.txt",
    samples_filename="samples.tsv",
    circuit_filename="circuit.txt",
)

Sweeping τ_c:   0%|          | 0/13 [00:00<?, ?τ_c/s]

samples @ τ_c=20.0:   0%|          | 0/500 [00:00<?, ?sample/s]

KeyboardInterrupt: 

#### [re] exp2

In [ ]:
from experiments import run_exp2_experiment, save_results_exp2, make_run_dir, load_config

# 1) 读取 YAML
cfg = load_config("exp2_rb.yaml")

# 2) 跑实验（返回结果 + 元数据）
results = run_exp2_experiment(cfg)

run_dir = make_run_dir(exp_name="experiment2")

# 3) 保存结果（带 header）

save_results_exp2(cfg, results, run_dir)

In [ ]:
import cirq, numpy as np
from typing import Dict, List, Tuple, Optional, Iterable, Any


import copy
import math
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime

# utils
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context
from cirq.noise.utils.composite_noise_model import CompositeNoiseModel
from cirq.noise.utils.randomized_benchmarking import _survival_prob_from_dm, FitResult, fit_rb_decay, build_rb1_circuit, DEFAULT_RB_GATE_SET, clifford_1q_gate_set, fill_rb_gate_samplers, _run_rb_sweep_custom
from cirq.noise.utils.metrics import fidelity

# builder（延迟导入模型）
from cirq.noise.utils.noise_builder import (
    _get,
    eval_number,
    make_noise_model
)

from cirq.noise.models.segmented_flux_noise_model import SegmentedFluxNoiseModel, sample_flux_noise_segments

from cirq.noise import IdleNoiseModel, RyGateNoiseModel, PhotonDecayNoiseModel

from cirq.noise.utils.compilation_scheme import all_in_one_compile

rng_seq = np.random.RandomState()
qs = cirq.LineQubit.range(3)
c = build_rb1_circuit(
                qs, depth = 20,
                gate_set = clifford_1q_gate_set(),
                seed = rng_seq.randint(2**31),
                measure = False
            )

print(f"initial circuit:\n{c}")

c1 = all_in_one_compile(c)
print(f"compiled circuit:\n{c1}")
sim_noisy = cirq.DensityMatrixSimulator()

context = assign_timed_circuit_context(c1, tau_c=60, treat_z_as_virtual = False)

max_seg = max(info.segment_id for infos in context.timing_map.values() for info in infos)
all_qubits = sorted(c1.all_qubits(), key=lambda q: getattr(q, "x", str(q)))
delta_phis = sample_flux_noise_segments(all_qubits, max_seg)

model1 = IdleNoiseModel(treat_z_as_virtual = False)
model2 = PhotonDecayNoiseModel(ctx = context)
model3 = RyGateNoiseModel()
model4 = SegmentedFluxNoiseModel(timing_map = context.timing_map, delta_phis= delta_phis)

composite_model = CompositeNoiseModel([model1, model2, model3, model4])

c_noisy = c1.with_noise(composite_model)
rho_noisy = sim_noisy.simulate(c_noisy).final_density_matrix
print(rho_noisy)

#### [re] exp2_irb

In [ ]:
from experiments import (
    run_interleaved_rb_experiment,
    save_results_interleaved_rb,
    make_run_dir,
    load_config,
)

# === 1. 读取 YAML 配置 ===
cfg = load_config("exp2_interleaved_rb.yaml")

# === 2. 执行实验（包含参考 + 每个门的 interleaved）===
results = run_interleaved_rb_experiment(cfg)

# === 3. 创建运行输出目录（默认存入 results/exp2_interleaved_rb/下）===
run_dir = make_run_dir(exp_name="exp2_interleaved_rb")

# === 4. 保存结果文本文件（results.txt）===
save_results_interleaved_rb(cfg, results, run_dir)

#### yikang's sim

In [15]:
import cirq
from typing import Sequence
import numpy as np

def thermal_channel(
    qubit:cirq.Qid,
    duration:float = None,
    T1:float = 30e-6,
    T2:float = 30e-6
    ) -> Sequence[cirq.KrausChannel]:
    
    Tphi = 1 / (1 / T2 - 1 / (2 * T1))
    
    p1 = 1.0 - np.exp(- duration / T1)
    pphi = 1.0 - np.exp(- duration / Tphi)

    return [cirq.amplitude_damp(p1).on(qubit), cirq.phase_damp(pphi).on(qubit)]

def circuit_dt(dt:float = 1e-9)->cirq.Circuit:
    q0, q1 = cirq.LineQubit.range(2)
    return cirq.Circuit(
    cirq.H(q0),
    cirq.H(q1),
    thermal_channel(q0, 135 * dt),
    thermal_channel(q1, 135 * dt),
    thermal_channel(q0, 225 * dt),
    thermal_channel(q1, 225 * dt),
    thermal_channel(q1, 270 * dt),
    cirq.measure(q0, q1)
    )

print(circuit_dt(dt = 1e-9))

sim = cirq.Simulator()

result = sorted(sim.run(circuit_dt(), repetitions=100000).histogram(key='q(0),q(1)').items())

print(dict(result))

0: ───H───AD(0.00449)───PD(0.00225)───AD(0.00747)───PD(0.00374)───────────────────────────────M───
                                                                                              │
1: ───H───AD(0.00449)───PD(0.00225)───AD(0.00747)───PD(0.00374)───AD(0.00896)───PD(0.00449)───M───
{0: 26021, 1: 24750, 2: 25010, 3: 24219}
